# <font color='black'> Shoot movements as a feature for detecting stress</font>
This research aimed to detect and measure leaf movement and displacement of A. thaliana seedlings using the Lucas-Kanade optical flow method. It estimated the leaf expansion and movement from a vector field obtained from consecutive images of the plants taken from the top view.<br/>

### Import &rarr; Python modules
The Shoot movements program needs multiple Python modules to run properly

In [ ]:
import os
import sys
import cv2
import datetime
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from matplotlib.ticker import PercentFormatter

from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy.fft import fft, fftfreq

### Import &rarr; User modules
We created our own modules to organize the code and follow the object oriented programming philosophy

In [ ]:
import plotting
import imalgo

### Variable declaration 
Define the Experiment & camera ID details

In [ ]:
expID, daySowing, cam = "exp09", "2018-09-11-00-00", "cam03"  # Color Constancy
dayTreat = "2018-10-02-17-30"
dateSowing = datetime.datetime.strptime(daySowing, "%Y-%m-%d-%H-%M")
dateTreat = datetime.datetime.strptime(dayTreat, "%Y-%m-%d-%H-%M")

### Directories 
Define the folder paths where is located the green fabric dataset

In [ ]:
dirCurrent = os.getcwd()
dirParent = os.path.abspath(os.path.join(dirCurrent, os.pardir)) 
coor_path = os.path.join(dirParent, 'Datasets', expID, expID + "_coord_" + cam + ".csv")
folder_images   = os.path.join(dirParent, 'Datasets', expID, expID + '_' + 'img')
csv_folder = os.path.join(dirParent, 'Datasets', expID, expID + '_' + "csv_measures")


if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

### Read files & set Variables
Parameters for the image prespective and lens

In [ ]:
epsilon = np.finfo(float).eps
coor_df = pd.read_csv(coor_path)
outliers = coor_df.loc[coor_df["outlier"] == "yes", "position"].tolist()

img_folders = [x for x in os.listdir(folder_images) if x != ".DS_Store"]

df_folders = pd.DataFrame(img_folders, columns=["folderName"])
df_folders[["position", "name", "treatment"]] = df_folders["folderName"].str.split('_',expand=True)

df_filter = df_folders[~df_folders["position"].isin(outliers)]
df_filter.reset_index(drop=True, inplace=True)

winSiz = 295
step = 1
vec_ang = np.arange(-180, 180 + step, step, dtype=int)
new_cols = ["area_prev", "area_next", "mag_mean", "mag_sum"] + np.copy(
    vec_ang
).astype(str).tolist()

### Good features and Lucas-Kanade optical flow parameters

In [ ]:
feature_params = dict(
    maxCorners=1000, qualityLevel=0.3, minDistance=3, blockSize=3)

lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),)

### Optical flow calculation between consecutive images

In [ ]:
# for cnt1 in range(40, 41):
for cnt1 in range(len(df_filter)):
    df_output = pd.DataFrame()
    folder_name, posn, name, treat = df_filter.loc[cnt1, :].values
    dirImgs = os.path.join(folder_images, folder_name)
    list_images = [x for x in os.listdir(dirImgs) if (x[-3:]) == "png"]
    
    print(cnt1, folder_name, posn, name, treat)

    df_main = pd.DataFrame(list_images, columns=["filename"])
    df_main["date"] = df_main["filename"].apply(
        lambda x: datetime.datetime.strptime(x[6:-4], "%Y-%m-%d-%H-%M-%S")
    )
    df_main["DAS"] = df_main["date"].apply(lambda x: (x - dateSowing).days)
    days = df_main.loc[:, "DAS"].unique()
    
    for das in days:
        
        # if das != 19: continue

        df_das = df_main[(df_main["DAS"] == das)]
        df_das = df_das.dropna(axis=0)
        df_das.reset_index(drop=True, inplace=True)
        df_das = df_das.reindex(columns=df_das.columns.tolist() + new_cols)
        
        path_prev = os.path.join(dirImgs, df_das.loc[0, "filename"])
        img_prev, gray_prev, area_prev = imalgo.gray_img(winSiz, path_prev)
        p0 = cv2.goodFeaturesToTrack(gray_prev, mask=None, **feature_params)

        for cnt3 in range(len(df_das)-1):
            path_next = os.path.join(dirImgs, df_das.loc[cnt3 + 1, "filename"])
            img_next, gray_next, area_next = imalgo.gray_img(winSiz, path_next)
                       
            if len(gray_next)== 0 : 
                df_das.loc[cnt3, "area_next"] = area_prev
                df_das.loc[cnt3, "area_prev"] = area_prev
                df_das.loc[cnt3, "mag_mean"] = mag_mean
                df_das.loc[cnt3, "mag_sum"] = mag_sum
                continue
            
            p1, st, err = cv2.calcOpticalFlowPyrLK(gray_prev, gray_next, p0, None, **lk_params)
            
            if p1 is not None:
                good_new = p1[st == 1]  # Select good points
                good_old = p0[st == 1]
                
            pts_diff = good_new - good_old
            pts_mag = np.sqrt(pts_diff[:, 0] ** 2 + pts_diff[:, 1] ** 2)

            pts_diff[pts_diff[:, 1] == 0, 1] = epsilon
            pts_ang = np.int16(
                np.round(np.arctan(pts_diff[:, 0] / pts_diff[:, 1]) * 180 / np.pi)
            )
           
            img_next[np.where((img_next == [0, 0, 0]).all(axis=2))] = [255, 255, 255]
            
            # %%
            img_prev = img_next.copy()
            gray_prev = gray_next.copy()
            area_prev = area_next
            p0 = good_new.reshape(-1, 1, 2)

            #%%
            df_das.loc[cnt3, "area_next"] = area_next
            df_das.loc[cnt3, "area_prev"] = area_prev

            mag_sum = np.sum(pts_mag)
            mag_mean = 0

            if mag_sum != 0:
                mag_mean = np.mean(pts_mag)

            df_das.loc[cnt3, "mag_mean"] = mag_mean
            df_das.loc[cnt3, "mag_sum"] = mag_sum
            
        df_output = pd.concat([df_output, df_das.loc[0:len(df_das)- 2,: ]])
        df_output.reset_index(drop=True, inplace=True)

    
    df_output.to_csv(os.path.join(csv_folder, folder_name + ".csv"), index=False)
    

### Merging data

In [ ]:
csv_output = os.path.join(dirParent, 'Datasets', expID, expID + '_' + "csv_output")
coor_df = pd.read_csv(coor_path)

list_csv = [x for x in os.listdir(csv_folder) if (x[-3:]) == "csv"]
df_csv = pd.DataFrame(list_csv, columns=["filename"])
df_csv[["position","name", "treatment"]] = df_csv["filename"].str.split('_',expand=True)
df_csv["treatment"] = df_csv["treatment"].apply(lambda x: x[:-4])
outliers = coor_df.loc[coor_df["outlier"] == "yes", "position"].tolist()

if not os.path.exists(csv_output):
    os.makedirs(csv_output)

In [ ]:
ref = 0
min_len = 100
pixelSiz = 0.235

for ecotype in df_csv.loc[:, "name"].unique():
    
    for treat in df_csv.loc[:, "treatment"].unique():
        
        df_rre = pd.DataFrame()
        df_disp = pd.DataFrame()
        
        df_aux = df_csv[(df_csv["treatment"]==treat) & (df_csv["name"]==ecotype)].sort_values(by="position")
        df_aux.reset_index(drop=True, inplace=True)
        
        
        for cnt in range(len(df_aux)):
            timeFit, areaF, param, perr = [], [], [], []
            filename, posn = df_aux.loc[cnt, ["filename", "position"]]
            file_csv = pd.read_csv(os.path.join(csv_folder, filename))

            if cnt == ref:
                df_rre["date"] = file_csv.loc[:, "date"]
                
                df_rre["mins"] = file_csv["date"].apply(lambda x: (datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S") - dateSowing
                ).total_seconds()/ 60.0)
                df_rre["das"] = df_rre.loc[:, "mins"]/(60*24)
                df_disp = df_rre.copy()
            
            if len(file_csv) != len(df_rre):             
                continue
            
            timeFit, areaF = imalgo.fitting(df_rre.loc[:, "mins"].values, file_csv.loc[:, "area_prev"].values)
            df_rre[posn] = (file_csv.loc[:, "area_prev"].values - areaF) * pixelSiz          
            df_disp[posn] = file_csv.loc[:, "mag_sum"] * pixelSiz
            
            _das = np.unique(np.int16(np.floor(df_disp.loc[:, "das"].values)))
            for cnt1 in range(len(_das) - 1):
                df_das = []
                df_das = df_disp.loc[(df_disp["das"] >_das[cnt1]) & (df_disp["das"] <_das[cnt1 +1]),  posn]
                std_thr =  4 * df_das.std()
                df_das[df_das > std_thr] = std_thr
                df_disp.loc[(df_disp["das"] >_das[cnt1]) & (df_disp["das"] <_das[cnt1 +1]),  posn] = df_das.values
            
            name_out = ecotype + "_" + treat + "_"
            df_rre.to_csv(os.path.join(csv_output, name_out + "Rosette relative expansion"+ ".csv"), index=False)
            df_disp.to_csv(os.path.join(csv_output, name_out + "Displacement"+ ".csv"), index=False)
            